In [ ]:
"""
CPU: AMD Ryzen 9 5950x
GPU: Nvidia Tesla M40 24G
"""

In [ ]:
import sys
sys.path.append("..")
from utils import *
from models_torch import *

In [10]:
device = torch.device('cuda')
valid_tape = "Z:/rppg/ccnu_datatape_160x128x128_valid.h5"
valid = load_datatape(valid_tape)
gra_sharp = 2.
physformer = ViT_ST_ST_Compact3_TDC_gra_sharp(image_size=(160,128,128), patches=(4,4,4), dim=96, ff_dim=144, num_heads=4, num_layers=12, dropout_rate=0.1, theta=0.7).to(device)
physformer.load_state_dict(torch.load('../weights/Physformer.pkl'))
physformer.train()
n, t = 0, 0
with torch.no_grad():
    for i, j in valid:
        _ = time.time()
        physformer(torch.tensor(np.array([i[..., [2, 1, 0]].transpose(3, 0, 1, 2)])).to(device).float()*2-1, gra_sharp)
        t += time.time()-_
        n += 1
print(f'{t/n/160*1000:.2f}ms per frame for PhysFormer on GPU')

0.39ms per frame for PhysFormer on GPU


In [15]:
for i, j in valid:
    break
i = np.array([i[..., [2, 1, 0]].transpose(3, 0, 1, 2)])
np.save('tmp.npy', i)

In [16]:
device = torch.device('cpu')
valid_tape = "Z:/rppg/ccnu_datatape_160x128x128_valid.h5"
valid = load_datatape(valid_tape)
gra_sharp = 2.
physformer = ViT_ST_ST_Compact3_TDC_gra_sharp(image_size=(160,128,128), patches=(4,4,4), dim=96, ff_dim=144, num_heads=4, num_layers=12, dropout_rate=0.1, theta=0.7).to(device)
physformer.load_state_dict(torch.load('../weights/Physformer.pkl'))
physformer.train()
n, t = 0, 0
with torch.no_grad():
    for i, j in valid:
        _ = time.time()
        physformer(torch.tensor(np.array([i[..., [2, 1, 0]].transpose(3, 0, 1, 2)])).to(device).float()*2-1, gra_sharp)
        t += time.time()-_
        n += 1
print(f'{t/n/160*1000:.2f}ms per frame for PhysFormer on CPU')

3.97ms per frame for PhysFormer on CPU


In [ ]:
import sys
sys.path.append("..")
from utils import *
from models import *

In [48]:
ts_can = keras.models.load_model('weights/TS-CAN_2022_CCNU')
valid_tape = to_tf(load_datatape("Z:/rppg/ccnu_datatape_160x36x36_valid.h5"))
n, t = 0, 0
for i, j in valid_tape:
    _ = time.time()
    ts_can(i)
    t += time.time()-_
    n += 1
print(f'{t/n/160*1000:.2f}ms per frame for TS-CAN on GPU')

n, t = 0, 0
with tf.device('/CPU:0'):
    for i, j in valid_tape:
        _ = time.time()
        ts_can(i)
        t += time.time()-_
        n += 1
print(f'{t/n/160*1000:.2f}ms per frame for TS-CAN on CPU')

0.07ms per frame for TS-CAN on GPU
0.62ms per frame for TS-CAN on CPU


In [44]:
deep_phys = keras.models.load_model('weights/DeepPhys_CCNU')
def to_tf_deep_phys(datatape, dtype=tf.float16):
    def _():
        for i, j in datatape:
            yield i, tf.concat([j[1:]-j[:-1], [0]], axis=0)
    return tf.data.Dataset.from_generator(lambda :_(), output_types=(dtype, dtype), output_shapes=(datatape.shape, datatape.shape[:1]))
valid_tape = to_tf_deep_phys(load_datatape("Z:/rppg/ccnu_datatape_160x36x36_valid.h5"))
n, t = 0, 0
for i, j in valid_tape:
    _ = time.time()
    deep_phys(i)
    t += time.time()-_
    n += 1
print(f'{t/n/160*1000:.2f}ms per frame for DeepPhys on GPU')

n, t = 0, 0
with tf.device('/CPU:0'):
    for i, j in valid_tape:
        _ = time.time()
        deep_phys(i)
        t += time.time()-_
        n += 1
print(f'{t/n/160*1000:.2f}ms per frame for DeepPhys on CPU')

0.04ms per frame for DeepPhys on GPU
0.39ms per frame for DeepPhys on CPU


In [45]:
physnet = keras.models.load_model('weights/PhysNet_CCNU', {'np_loss':np_loss})
valid_tape = to_tf(load_datatape("Z:/rppg/ccnu_datatape_128x32x32_valid.h5"))
n, t = 0, 0
for i, j in valid_tape:
    _ = time.time()
    physnet([i])
    t += time.time()-_
    n += 1
print(f'{t/n/128*1000:.2f}ms per frame for PhysNet on GPU')

n, t = 0, 0
with tf.device('/CPU:0'):
    for i, j in valid_tape:
        _ = time.time()
        physnet([i])
        t += time.time()-_
        n += 1
print(f'{t/n/128*1000:.2f}ms per frame for PhysNet on CPU')


0.03ms per frame for PhysNet on GPU
0.31ms per frame for PhysNet on CPU


In [46]:
ours = keras.models.load_model('weights/m1')
valid_tape = to_tf(load_datatape("Z:/rppg/ccnu_datatape_450x8x8_valid.h5"))
n, t = 0, 0
for i, j in valid_tape:
    _ = time.time()
    ours([i])
    t += time.time()-_
    n += 1
print(f'{t/n/450*1e6:.2f}μs per frame for ours on GPU')

n, t = 0, 0
with tf.device('/CPU:0'):
    for i, j in valid_tape:
        _ = time.time()
        ours([i])
        t += time.time()-_
        n += 1
print(f'{t/n/450*1e6:.2f}μs per frame for ours on CPU')

5.82μs per frame for ours on GPU
27.89μs per frame for ours on CPU


In [49]:
ours_large_input = keras.models.load_model('weights/m4')
valid_tape = to_tf(load_datatape("Z:/rppg/ccnu_datatape_450x32x32_valid.h5"))
n, t = 0, 0
for i, j in valid_tape:
    _ = time.time()
    ours_large_input([i])
    t += time.time()-_
    n += 1
print(f'{t/n/450*1e6:.2f}μs per frame for ours_large_input on GPU')

n, t = 0, 0
with tf.device('/CPU:0'):
    for i, j in valid_tape:
        _ = time.time()
        ours_large_input([i])
        t += time.time()-_
        n += 1
print(f'{t/n/450*1e6:.2f}μs per frame for ours_large_input on CPU')

6.56μs per frame for ours_large_input on GPU
46.42μs per frame for ours_large_input on CPU
